<a href="https://colab.research.google.com/github/jskaza/nfl-big-data-bowl-2023/blob/master/model_training_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import math
from sklearn.linear_model import LogisticRegression

2023-01-02 13:45:26.714421: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-02 13:45:28.331740: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-02 13:45:28.331863: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-02 13:45:31.786049: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
input_file_name = 'dataset_week1.json'
filepath = os.path.join(os.getenv('NFL_DATA'), input_file_name)
df = pd.read_json(filepath)

#df = pd.read_json("/content/drive/MyDrive/nfl-big-data-bowl-2023/data/dataset_week1.json")
df.head()

nfl_id                    opponent_x                        opponent_y  \
0   41263  [-2.61, -1.29, 0.1, 1.73, 3]  [0.45, 0.15, -0.54, -0.37, 0.18]   
1   42403  [3, 1.73, 0.1, -1.29, -2.61]  [0.18, -0.37, -0.54, 0.15, 0.45]   
2   44955  [1.73, 0.1, 3, -1.29, -2.61]  [-0.37, -0.54, 0.18, 0.15, 0.45]   
3   53441  [-1.29, 0.1, -2.61, 1.73, 3]  [0.15, -0.54, 0.45, -0.37, 0.18]   
4   53504  [1.73, 3, 0.1, -1.29, -2.61]  [-0.37, 0.18, -0.54, 0.15, 0.45]   

                   teammate_x                    teammate_y  ball_x  ball_y  \
0   [-1.99, 1.26, 2.75, 8.71]  [-2.12, -1.79, -2.14, -2.34]     0.0     0.0   
1  [2.75, 1.26, -1.99, -5.03]  [-2.14, -1.79, -2.12, -1.74]     0.0     0.0   
2  [2.75, -1.99, -5.03, 8.71]  [-2.14, -2.12, -1.74, -2.34]     0.0     0.0   
3   [-5.03, 1.26, 2.75, 8.71]  [-1.74, -1.79, -2.14, -2.34]     0.0     0.0   
4  [1.26, -1.99, 8.71, -5.03]  [-1.79, -2.12, -2.34, -1.74]     0.0     0.0   

      game_id  play_id  frame_id  ...  pff_sack  pff_position_lined_up  \
0  2021090900       97         6  ...         0                    LEO   
1  2021090900       97         6  ...         0                   ROLB   
2  2021090900       97         6  ...         0                    DRT   
3  2021090900       97         6  ...         0                   LILB   
4  2021090900       97         6  ...         0                     RE   

   quarter  down  yards_to_go absolute_yardline_number       personnel_o  \
0        1     3            2                       43  1 RB, 1 TE, 3 WR   
1        1     3            2                       43  1 RB, 1 TE, 3 WR   
2        1     3            2                       43  1 RB, 1 TE, 3 WR   
3        1     3            2                       43  1 RB, 1 TE, 3 WR   
4        1     3            2                       43  1 RB, 1 TE, 3 WR   

   defenders_in_box  offense_formation  score_delta  
0               6.0            SHOTGUN            0  
1               6.0            SHOTGUN            0  
2               6.0            SHOTGUN            0  
3               6.0            SHOTGUN            0  
4               6.0            SHOTGUN            0  

[5 rows x 24 columns]

In [3]:
def make_features(df: pd.DataFrame, feats: list, outcomes: list):
    X, y  = [], []
    grouped_df = df.groupby(["nfl_id","game_id","play_id"])
    for group_name, group_df in grouped_df:
      X.append(group_df[feats].to_numpy())
      y.append(group_df[outcomes].to_numpy()[0])
    
    X = tf.keras.utils.pad_sequences(X ,dtype="float", padding="pre", value= -99)
    y = tf.keras.utils.pad_sequences(y ,dtype="float", padding="pre", value= -99)
    return X, y

In [4]:
feats = ["x","y","ball_x","ball_y"]
outcomes = ["pff_hurry"]
X, y = make_features(df, feats, outcomes)

split = 0.8
num_train = round(split * X.shape[0])

X_train = X[:num_train]
y_train = y[:num_train]

X_test = X[num_train:]
y_test = y[num_train:]


In [5]:
# set random seed for reproducibility

tf.random.set_seed(42)
np.random.seed(42)
NUM_EPOCHS = 5

# Our Model

In [6]:


# create the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Masking(mask_value=-99.,input_shape= X.shape[1:]))
model.add(tf.keras.layers.LSTM(32, input_shape = X.shape[1:]))
model.add(tf.keras.layers.Dense(y.shape[1], activation="sigmoid"))

# compile and fit the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics = [tf.keras.metrics.AUC()])

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

2023-01-02 13:45:49.949822: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-02 13:45:49.950034: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-02 13:45:49.950186: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (penguin): /proc/driver/nvidia/version does not exist
2023-01-02 13:45:50.478243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
48/48 [==============================] - 10s 118ms/step - loss: 0.4187 - auc: 0.5499 - val_loss: 0.2591 - val_auc: 0.6927
Epoch 2/5
48/48 [==============================] - 4s 91ms/step - loss: 0.2580 - auc: 0.7288 - val_loss: 0.2461 - val_auc: 0.7948
Epoch 3/5
48/48 [==============================] - 4s 88ms/step - loss: 0.2435 - auc: 0.8020 - val_loss: 0.2361 - val_auc: 0.8185
Epoch 4/5
48/48 [==============================] - 4s 89ms/step - loss: 0.2314 - auc: 0.8250 - val_loss: 0.2302 - val_auc: 0.8303
Epoch 5/5
30/30 [==============================] - 1s 31ms/step - loss: 0.1939 - auc: 0.8250


[0.19389042258262634, 0.8249914646148682]

# Modified Transformer Example

In [15]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [18]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = layers.Masking(mask_value=-99.,input_shape= X.shape[1:])(x)
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(y.shape[1], activation="sigmoid")(x)
    return keras.Model(inputs, outputs)

In [19]:
input_shape = X_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[64],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics = [tf.keras.metrics.AUC()]
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=NUM_EPOCHS,
    #batch_size=64,
    callbacks=callbacks,
)

model.evaluate(X_test, y_test, verbose=1)

Epoch 1/5
96/96 [==============================] - 100s 972ms/step - loss: 0.8535 - auc_2: 0.5258 - val_loss: 0.7509 - val_auc_2: 0.5424
Epoch 2/5
96/96 [==============================] - 94s 979ms/step - loss: 0.8119 - auc_2: 0.5091 - val_loss: 0.7254 - val_auc_2: 0.5383
Epoch 3/5
93/96 [============================>.] - ETA: 4s - loss: 0.7757 - auc_2: 0.5106

KeyboardInterrupt: 

# Transformer Example

In [10]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [11]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [12]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences
(25000, 200)
(25000,)


In [13]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [14]:
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
# history = model.fit(
#     x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
# )

Epoch 1/2
 48/782 [>.............................] - ETA: 1:19 - loss: 0.6937 - accuracy: 0.5410

KeyboardInterrupt: 